In [ ]:
import matplotlib.pyplot as plt
import bproc
import tqdm
import btrack
import h5py as h5
import numpy as np

### For the OCD folks (like me), set some nice matplotlib plotting parameters

# Set some rcparams
# plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['font.size'] = 8
plt.rcParams['font.family'] = 'serif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=[
    'k', 'b', 'c', 'r', 'm'
])



import btrack
import tqdm
import matplotlib.pyplot as plt
import cv2
import bproc
from IPython import embed
import os
import h5py as h5
import numpy as np

import numpy as np
import numpy as np
import time


# Load the confg path
config_path = '/home/sunnyliu1220/git/latent-gaze/config.yaml'
           

config = bproc.load_proc_config(config_path)

# Pull out high-level config info
paths = config["paths"]
params = config["params"]
reconstruction_params = config['reconstruction']

use_color = reconstruction_params['use_color']
print(use_color)
print(reconstruction_params['crop_dim'][:])

### Copy the created h5 to a new location with appropriate name

# Copy the destination file to a new file with a different name
destination = paths["destination"]
rec_destination = destination.replace(".h5", "_reconstructed.h5")

if os.path.exists(rec_destination):
    print('RECONSTRUCTED FILE DESTINATION EXISTS - MOVE IT OR IT WILL BE OVERWRITTEN')

    

print(f"Deleting {rec_destination}")
print(f"Copying {destination} to {rec_destination}")

os.system(f"rm {rec_destination}")
print(destination)
print(destination)
print(destination)
os.system(f"cp {destination} {rec_destination}")

file = h5.File(rec_destination, 'a')



In [ ]:
### Load the file and determine pixel-shifts for entire experiment

total_gaze, mean_gaze, min_gaze, max_gaze, mp4_frame_idxs = bproc.get_gaze_statistics(rec_destination)
concat_elevation, concat_azimuth = total_gaze
mean_elevation, mean_azimuth = mean_gaze
min_elevation, min_azimuth = min_gaze
max_elevation, max_azimuth = max_gaze

reconstruction_parameters = file['meta']['reconstruction']

full_dim = file['meta']['reconstruction']['full_dim'][:]


print(mp4_frame_idxs.shape)

dy, dx = bproc.calc_pixel_shifts(total_gaze[0], total_gaze[1], full_dim)
print(dx.shape, dy.shape)

reconstruction_parameters['crop_dim'][:]

shifted = True
static = False

reconstruction_parameters['lower_left'][:]=[90,115]

which_series = 'all'
# which_series = ['series_004']
if which_series == 'all':
    which_series = file['data'].keys()



In [ ]:
for series in tqdm.tqdm(which_series):
    print('~~~~~~!!!~~~~~~')
    print('Processing series ', series)
    for epoch in file['data'][series].keys():
        print('+++++++ Processing epoch ',epoch)
        epoch_data = file['data'][series][epoch]

        stim_type = epoch_data.attrs['stim_type']

        epoch_elevation = epoch_data['signals']['interp_elevation'][:]
        epoch_azimuth = epoch_data['signals']['interp_azimuth'][:]

        relative_elevation = epoch_elevation - min_elevation
        relative_azimuth = max_azimuth - epoch_azimuth

        dy, dx = bproc.calc_pixel_shifts(relative_elevation, relative_azimuth, full_dim)
        epoch_data['signals'].create_dataset('dy', data=dy)
        epoch_data['signals'].create_dataset('dx', data=dx)

        epoch_data.create_group('stimulus')

        if shifted:
            shift_stim=True
            if stim_type == 'white_noise':
                stim = bproc.reconstruct_whitenoise(epoch_data, reconstruction_parameters, shift_stim, use_color=use_color)
            elif stim_type == 'movie':
                stim = bproc.reconstruct_movie(epoch_data, reconstruction_parameters, paths['stimulus_mp4'], shift_stim, use_color=use_color)
            epoch_data.create_dataset('stimulus/shifted', data=stim)
            print('++++++++++++ saved shifted')

        if static:       
            shift_stim=False
            if stim_type == 'white_noise':
                stim = bproc.reconstruct_whitenoise(epoch_data, reconstruction_parameters, shift_stim, use_color=use_color)
            elif stim_type == 'movie':
                stim = bproc.reconstruct_movie(epoch_data, reconstruction_parameters, paths['stimulus_mp4'], shift_stim, use_color=use_color)
    
            epoch_data.create_dataset('stimulus/static', data=stim)
            print('++++++++++++ saved static')


file.close()